In [1]:
import pandas as pd 
import json
import gzip
import shutil
import glob 
from langdetect import detect 

In [2]:
train_files = sorted(glob.glob("data/papers/s2-corpus-*.gz"))

In [17]:
import pandas as pd 
import json
import gzip
import shutil
import glob 

train_files = sorted(glob.glob("data/papers/s2-corpus-*.gz"))

lines = []
# Load dataframe for all papers 
for filepath in train_files[:1]:
    print(filepath)
    with gzip.open(filepath, 'rb') as f_in:
        print(f_in)

        # unzip, but not necessary 
        # with open(filepath.strip('.gz'), 'wb') as f_out:
        #     shutil.copyfileobj(f_in, f_out)

        for cnt, line in enumerate(f_in):
            try: 
                lines.append(json.loads(line))
            except: # any line errors 
                pass 

print('read in {} entities'.format(len(lines)))

data/papers/s2-corpus-002.gz
<gzip _io.BufferedReader name='data/papers/s2-corpus-002.gz' 0x1182671d0>
read in 996168 entities


In [70]:
# train_df = pd.DataFrame.from_dict(lines)

In [74]:
df = train_df.head(20)

In [78]:
# Filter out non-english titles (and therefore abstracts)
df = df[[detect(i) =='en' for i in df.title]]

In [41]:
# # Can use improving, 
# # but filters by assuming that if > 50% of the title chars are nonEnglish, 
# # then the paper is not in english
# def isEnglish(s):    
#     non_eng = 0
#     for i in s:         
#         try:
#             i.encode(encoding='utf-8').decode('ascii')
#         except UnicodeDecodeError:
#             non_eng += 1 
#     if non_eng/len(s) > 0.5: 
#         return False
#     return True

In [122]:
def save_authors_df(dataframe, outdir):
    df = dataframe
    authors = []
    for i in df.authors:
        authors.extend(i)
        
    author_df = pd.DataFrame.from_dict(authors)
    
    author_df = author_df.dropna(subset=['ids'])
    author_df.ids = author_df.ids.str[0]
    
    author_df = author_df.dropna() # probably unnecessary 
    author_df.ids.iloc[:] = author_df.ids.astype(int)
    
    author_df.to_csv(outdir)

In [114]:
author_df = pd.DataFrame.from_dict(authors)

In [115]:
author_df = author_df.dropna(subset=['ids'])
author_df.ids = author_df.ids.str[0]



In [123]:
save_authors_df(df, 'data/papers/authors.csv')

In [117]:
author_df.shape

(50, 2)

In [118]:
author_df = author_df.dropna()

In [119]:
author_df.ids.iloc[:] = author_df.ids.astype(int)

In [120]:
author_df.head()

,ids,name
0,91488677,Zeno Atherton
1,92191577,Charlotte Waveney Faulkner
2,91884879,Scott L. Ingham
3,39854403,Ashok K. Kakkar
4,50991275,Muhammad Saif Ullah Khan


In [121]:
author_df.dtypes

ids      int64
name    object
dtype: object